### 코드 전체 흐름
1. 문서의 내용을 읽는다
2. 문서를 쪼갠다.
- 토큰수 초과로 답변을 생성하지 못할 수 있고
- 문서가 길면(Input이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 DB에 저장
4. 질문이 있을 때, 벡터 DB에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [1]:
%pip install --upgrade --quiet  docx2txt langchain-community
%pip install -qU langchain-text-splitters
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [16]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,    # 문서를 쪼갤 때 하나의 chunk 가 가지는 토큰 수
    chunk_overlap=200,  # 텍스트를 분할할 때, 각 chunk(조각) 사이에 중복되는 부분의 토큰 수
)

loader = Docx2txtLoader('./tax_with_markdown.docx')
document = loader.load()                                           #단순히 문서를 읽을 때
document_list = loader.load_and_split(text_splitter=text_splitter) # 문서를 쪼갤 때

In [17]:
print('쪼개기 전 문서 : ',len(document))
print('쪼갠 문서 : ',len(document_list))

쪼개기 전 문서 :  1
쪼갠 문서 :  225


In [19]:
from dotenv import load_dotenv                # OpenAI API Key 사용을 위해 환경변수 등록
from langchain_openai import OpenAIEmbeddings # OpenAI 의 Embedding을 사용

load_dotenv() # 환경변수 불러오기

embeddings = OpenAIEmbeddings(model='text-embedding-3-large') # 기본모델은 002인 예전모델이다. 신규 모델을 사용하기 위해 추가

In [20]:
%pip install -qU langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


In [21]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-markdown-index'
pinecone_api_key = os.environ.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)
database = PineconeVectorStore.from_documents(document_list, embeddings, index_name=index_name)

In [46]:
query = '연봉 3000만원인 직장인의 소득세는 얼마인가요?'

In [47]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [48]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

C:\Users\DataLogics\miniconda3\envs\llm_lab\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [49]:
retriever = database.as_retriever(search_kwargs={'k':2})
retriever.invoke(query)

[Document(id='7343a37d-86e6-43a7-873e-8bedef4fc5c5', metadata={'source': './tax_with_markdown.docx'}, page_content='1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n\n다. 대통령령으로 정하는 농어가부업소득\n\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 급여\n\n나. 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n\n다. 「산업재해보상보험법」에 따라 수급권자가 받는 요양급여, 휴업급여, 장해급여, 간병급여, 유족급여, 유족특별급여, 장해특별급여, 장의비 또는 근로의 제공으로 인한 부상ㆍ질병ㆍ사망과 관련하여 근로자나 그 유족이 받는 배상ㆍ보상 또는 위자(慰藉)의 성질이 있는 급여\n\n라. 「근로기준법」 또는 「선원법」에 따라 근로자ㆍ선원 및 그 유족이 받

In [50]:
# QA Chain 만들기
from langchain.chains import RetrievalQA


qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt":prompt}
)

# Query -> 직장인 -> 거주자 Chain 추가
ai_message = qa_chain({"query" : query})

In [51]:
ai_message

{'query': '연봉 3000만원인 직장인의 소득세는 얼마인가요?',
 'result': '죄송하지만 제공된 문맥에서는 연봉 3000만원인 직장인의 소득세를 계산하는 데 필요한 정보가 없습니다. 소득세 계산에는 과세표준과 세율 등의 구체적인 세법 정보가 필요합니다. 이러한 정보는 관련 세무법령이나 세무 전문가의 조언을 통해 얻을 수 있습니다.'}

In [52]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ['사람을 나타내는 표현 -> 거주자']

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
    사전: {dictionary}

    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [53]:
new_question = dictionary_chain.invoke({"question":query})
new_question

'질문: 연봉 3000만원인 거주자의 소득세는 얼마인가요?'

In [54]:
print("바꾸기 전 : "+query)
print("바꾼 후 : "+new_question)

바꾸기 전 : 연봉 3000만원인 직장인의 소득세는 얼마인가요?
바꾼 후 : 질문: 연봉 3000만원인 거주자의 소득세는 얼마인가요?


In [55]:
tax_chain = {"query" : dictionary_chain} | qa_chain

In [56]:
ai_response = tax_chain.invoke({"question":query})

In [57]:
ai_response

{'query': '연봉 3000만원인 거주자의 소득세는 얼마인가요?',
 'result': '연봉 3,000만원인 거주자의 소득세는 1,400만원을 초과하는 1,600만원에 대해 15%의 세율이 적용됩니다. 따라서, 1,400만원 이하에 대한 세금은 84만원이며, 나머지 1,600만원에 대한 세금은 240만원입니다. 최종 소득세는 84만원 + 240만원 = 324만원입니다.'}